## Test Interpretability techniques in AIS with BERT

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.special import softmax
from sklearn.metrics import f1_score, average_precision_score
from dataset import Dataset
from myModel import MyModel
from myExplainers import MyExplainer
from myEvaluation import MyEvaluation
from sklearn.preprocessing import maxabs_scale
import pickle
from tqdm import tqdm
import datetime
import csv
import torch
import warnings

In [2]:
data_path = '/models/'
model_path = '/models/'
save_path = '/results/AIS/'

In [3]:
model_name = 'bert'
existing_rationales = False

In [4]:
task = 'single_label'
sentence_level = False
labels = 2

model = MyModel(model_path, 'bert_ais', model_name, task, labels, 'cased')
max_sequence_len = model.tokenizer.max_len_single_sentence
tokenizer = model.tokenizer

In [5]:
ais = Dataset(path=data_path)
x, y, label_names = ais.load_AIS()
label_names = ['class a', 'class b']

In [6]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    x, y, test_size=.2, random_state=42)

size = (0.1 * len(y)) / len(train_labels)
train_texts, validation_texts, train_labels, validation_labels = train_test_split(
    list(train_texts), train_labels, test_size=size, random_state=42)


In [ ]:
predictions = []
for test_text in test_texts:
    outputs = model.my_predict(test_text)
    predictions.append(outputs[0])

#Binary
pred_labels = []
for prediction in predictions:
    pred_labels.append(np.argmax(softmax(prediction)))


def average_precision_wrapper(y, y_pred, view):
    return average_precision_score(y, y_pred.toarray(), average=view)


print(test_labels[:3])
print(pred_labels[:3])

print(average_precision_score(test_labels, pred_labels, average='macro'),
      f1_score(test_labels, pred_labels, average='macro'))


In [7]:
my_explainers = MyExplainer(label_names, model)
my_evaluators = MyEvaluation(label_names, model.my_predict, False)

In [9]:
def print_results(name, techniques, metrics):
    with open(name + '.csv', 'w', encoding='UTF8') as f:
        writer = csv.writer(f)
        for metric in metrics.keys():
            print(metric)
            temp_metric = np.array(metrics[metric])
            for i in range(len(techniques)):
                label_score = []
                for label in range(len(label_names)):
                    tempo = [
                        k for k in temp_metric[:, i, label]
                        if str(k) != str(np.average([]))
                    ]
                    if len(tempo) == 0:
                        tempo.append(0)
                    label_score.append(np.array(tempo))
                temp_mean = []
                for k in label_score:
                    temp_mean.append(k.mean())
                temp_mean = np.array(temp_mean).mean()
                writer.writerow(
                    [techniques[i], metric, temp_mean] +
                    [label_score[o].mean() for o in range(len(label_names))])
                print(
                    techniques[i], ' {} | {}'.format(
                        round(temp_mean, 5), ' '.join([
                            str(round(label_score[o].mean(), 5))
                            for o in range(len(label_names))
                        ])))


## Evaluating LIME and IG

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

    now = datetime.datetime.now()

    file_name = save_path + 'BERT_ais_LIME_IG_' + str(now.day) + '_' + str(
        now.month) + '_' + str(now.year)
    metrics = {'F': [], 'FTP': [], 'R': [], 'NZW': []}
    evaluation = {
        'F': my_evaluators.faithfulness,
        'FTP': my_evaluators.faithful_truthfulness_penalty,
        'R': my_evaluators.robustness,
        'NZW': my_evaluators.nzw
    }

    techniques = [my_explainers.lime, my_explainers.ig]
    for ind in tqdm(range(len(test_texts))):
        torch.cuda.empty_cache()
        instance = test_texts[ind]
        my_evaluators.clear_states()
        prediction, attention, hidden_states = model.my_predict(instance)
        enc = model.tokenizer([instance, instance],
                              truncation=True,
                              padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
        interpretations = []
        for technique in techniques:
            temp = technique(instance, prediction, tokens, mask, attention,
                             hidden_states)
            interpretations.append([maxabs_scale(i) for i in temp])
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _,
                                                    instance, prediction,
                                                    tokens, hidden_states, _,
                                                    _))
            k = k + 1
            metrics[metric].append(evaluated)

        with open(file_name + '.pickle', 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print_results(file_name, [' LIME', ' IG  '], metrics)

## Evaluation of Attention

In [24]:
conf = []
for ci in ['Mean', 'Multi', 'Sum'] + list( range(12)):  # Layers: Mean, Multi, Sum, First, Last
    for ce in ['Mean', 'Sum'] + list(range(12)):  #True every token, False only cls
        for cp in ['From', 'To', 'MeanColumns', 'MeanRows', 'MaxColumns','MaxRows']:  # Matrix: From, To, MeanColumns, MeanRows, MaxColumns, MaxRows
            for cl in [False]:  # Selection: True: select layers per head, False: do not
                conf.append([ci, ce, cp, cl])
for ci in ['Mean', 'Multi', 'Sum']:
    for ce in ['']:
        for cp in ['']:
            for cl in [True]:
                conf.append([ci, ce, cp, cl])
len(conf)

1263

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

    now = datetime.datetime.now()

    file_name = save_path + 'BERT_ais_TOKEN_Attention_' + str(
        now.day) + '_' + str(now.month) + '_' + str(now.year)
    metrics = {'F': [], 'FTP': [], 'NZW': []}
    evaluation = {
        'F': my_evaluators.faithfulness,
        'FTP': my_evaluators.faithful_truthfulness_penalty,
        'NZW': my_evaluators.nzw
    }

    for ind in tqdm(range(len(test_texts))):
        torch.cuda.empty_cache()
        instance = test_texts[ind]
        my_evaluators.clear_states()
        prediction, attention, hidden_states = model.my_predict(instance)
        enc = model.tokenizer([instance, instance],
                              truncation=True,
                              padding=True)[0]
        mask = enc.attention_mask
        tokens = enc.tokens
        interpretations = []
        for con in conf:
            my_explainers.config = con
            temp = my_explainers.my_attention(instance, prediction, tokens,
                                              mask, attention, hidden_states)
            interpretations.append([maxabs_scale(i) for i in temp])
        for metric in metrics.keys():
            evaluated = []
            k = 0
            for interpretation in interpretations:
                evaluated.append(evaluation[metric](interpretation, _,
                                                    instance, prediction,
                                                    tokens, hidden_states, _,
                                                    _))
            k = k + 1
            metrics[metric].append(evaluated)

        with open(file_name + '.pickle', 'wb') as handle:
            pickle.dump(metrics, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print_results(file_name, conf, metrics)